# Modélisation d’un problème de tournée sous contraintes
### Projet – Groupe 4 : Mathéo Pinget, Alban Calvo, Evan Joasson


## Contexte

L'objectif du projet est de proposer une solution algorithmique permettant d’optimiser les tournées de livraison ou de collecte dans un réseau routier, tout en respectant des contraintes pratiques fortes imposées par l’environnement réel :

- Certaines routes peuvent être coûteuses ou temporairement inutilisables.
- Certaines visites doivent impérativement être effectuées dans un ordre donné (exemple : livraison avant collecte).

Ce problème prend tout son sens dans une logique de mobilité durable et de logistique urbaine intelligente.


## Définition formelle

Le réseau routier est représenté par un **graphe orienté**, noté :

$$
G = (S, A)
$$

où :

---

### 1. Ensemble des sommets $S$

- $S = \{s_0, s_1, \dots, s_n\}$ représente l’ensemble des **villes à visiter**.
- Le sommet $s_0$ désigne le **dépôt**, c’est-à-dire le point de départ et de retour du véhicule.

**Exemple :**  
Si $n = 4$, alors $S = \{s_0, s_1, s_2, s_3, s_4\}$. Cela signifie qu’il y a **5 villes**, dont $s_0$ est le dépôt.  
Le véhicule commence et termine sa tournée à $s_0$.

---

### 2. Ensemble des arêtes $A$

- $A \subseteq S \times S$ est l’ensemble des **arcs orientés**, représentant les routes possibles entre les villes.
- Chaque arête $(i, j)$ indique qu’il est **possible de se rendre directement** de la ville $s_i$ à la ville $s_j$.
- Certaines routes peuvent être absentes du graphe si elles sont **interdites ou impraticables** (travaux, contraintes de circulation, etc.).

**Exemple :**  
Si  
$$
A = \{(s_0, s_1), (s_1, s_2), (s_2, s_3), (s_3, s_4), (s_4, s_0)\}
$$  
alors ces **5 connexions** sont les seules possibles dans la tournée. Le graphe n'est donc pas complet.

---

### 3. Coût associé à chaque arête $c_{ij}$

- À chaque arête $(i, j) \in A$ est associé un **coût** $c_{ij} \in \mathbb{R}^+$.
- Ce coût peut représenter :
  - la **distance** entre deux villes,
  - le **temps de trajet** estimé,
  - ou toute autre mesure d’impact (coût énergétique, émission de CO₂, etc.).

**Exemple :**  
Si $c_{12} = 10$, cela signifie qu’il faut **10 unités** (ex. : minutes ou kilomètres) pour aller de $s_1$ à $s_2$.

---

### 4. Contraintes de dépendance $D$

- $D \subseteq S \times S$ est l’ensemble des **contraintes de précédence**.
- Une contrainte $(i, j) \in D$ signifie que la ville $s_i$ **doit être visitée avant** la ville $s_j$.

**Exemple :**  
Si $(s_1, s_3) \in D$, alors $s_1$ doit être visité **obligatoirement avant** $s_3$.  
Cela modélise par exemple une **livraison à effectuer avant une collecte**.

---

### 5. Variables de décision

#### a) Variable binaire $x_{ij}$

- $x_{ij} \in \{0, 1\}$ indique si le véhicule **emprunte l’arête** $(i, j)$.
  - $x_{ij} = 1$ : le véhicule passe de $s_i$ à $s_j$.
  - $x_{ij} = 0$ : le véhicule ne passe pas par cette route.

**Exemple :**  
Si $x_{23} = 1$, cela signifie que le véhicule **effectue le trajet** de $s_2$ à $s_3$.

---

Cette modélisation permet de représenter un **système logistique réaliste** prenant en compte :  
- la topologie du réseau routier,  
- les coûts de transport,  
- les contraintes de parcours,  
- et les variables de décision pour une résolution algorithmique ultérieure.


## Objectif et contraintes

### Fonction objectif

L’objectif est de minimiser le **coût total de la tournée**, c’est-à-dire la somme des coûts des trajets réellement effectués par le véhicule.

$$
\min \sum_{(i, j) \in A} c_{ij} \cdot x_{ij}
$$

- $x_{ij} = 1$ si l’arête $(i, j)$ est empruntée (le véhicule passe de la ville $s_i$ à $s_j$).
- $c_{ij}$ représente le coût associé à cette arête : temps, distance, carburant, pollution, etc.

**Exemple :**  
Le véhicule emprunte deux arcs : $(s_1, s_3)$ et $(s_3, s_4)$.  
Si $c_{13} = 10$ (temps en minutes) et $c_{34} = 12$, alors le coût total est :  
$10 + 12 = 22$.

---

### Contraintes

---

#### 1. Chaque ville (hors dépôt) est visitée une seule fois

$$
\forall i \in S \setminus \{s_0\},\quad \sum_{j \in S} x_{ji} = 1 \quad \text{et} \quad \sum_{j \in S} x_{ij} = 1
$$

- Chaque ville $s_i$ (autre que le dépôt $s_0$) doit avoir **exactement une arête entrante et une arête sortante** dans la tournée.
- Cela garantit que le véhicule **passe une seule fois** par chaque ville.

**Exemple :**  
Pour $s_2$, on a une seule arête entrante, par exemple $(s_1, s_2)$, et une seule sortante, par exemple $(s_2, s_3)$.  
Donc $x_{12} = 1$ et $x_{23} = 1$.

---

#### 2. Départ et retour obligatoires au dépôt

$$
\sum_{j \in S} x_{s_0j} = 1 \quad \text{et} \quad \sum_{i \in S} x_{is_0} = 1
$$

- Le véhicule **quitte le dépôt $s_0$ une seule fois** et **revient au dépôt une seule fois**.

**Exemple :**  
Si la tournée commence par $(s_0, s_1)$ et se termine par $(s_4, s_0)$, alors on a :  
$x_{01} = 1$ et $x_{40} = 1$.

---

#### 3. Respect des dépendances entre visites

$$
ui < uj
$$

- Si la contrainte $(i, j)$ appartient à $D$, alors la ville $s_i$ **doit être visitée avant** $s_j$.

---

#### 4. Interdiction d’utiliser certaines arêtes

$$
\forall (i, j) \in A', \quad x_{ij} = 0
$$
$$
A' \in A
$$

- Si l’arête $(i, j)$ **n’existe pas** (route bloquée, interdite), elle **ne peut pas** être utilisée dans la tournée.

**Exemple :**  
Si $(s_2, s_4) \in A'$, alors $x_{24} = 0$.

---

Ces contraintes garantissent une tournée **valide et réaliste** :  
- Chaque ville est bien visitée,
- Les contraintes logistiques sont respectées,
- Les temps sont cohérents avec la progression du véhicule.


## Étude de complexité du problème

Ce problème généralise plusieurs variantes du Problème du Voyageur de Commerce (PVC), en ajoutant :

- des dépendances entre visites (précédence),
- des arêtes interdites ou à coût élevé.

Il s'agit d’un problème combinatoire avec un espace de recherche exponentiel.

### Taille de l’espace de recherche

Pour un graphe complet avec $n$ sommets :

- Le nombre de permutations possibles est de $(n - 1)!$ (le sommet de départ est fixé).
- Avec contraintes, certaines solutions sont interdites, mais l’ordre de grandeur reste $O(n!)$.

### Lien avec des problèmes NP-complets

| Problème associé | Complexité |
|------------------|------------|
| PVC (classique) | NP-complet |
| PVC avec précédences | NP-complet |
| PVC avec arêtes interdites | NP-complet |
| PVC avec contraintes multiples | NP-difficile |


### Appartenance à NP

La solution peut être vérifiée en temps polynomial :

- Vérification des dépendances : $O(|D|)$
- Vérification des arêtes utilisées : $O(n^2)$
- Calcul du coût : $O(n)$

Donc, le problème est dans la classe NP.

### Difficulté d’approximation

- Le PVC est APX-inapproximable.
- En présence de précédences et d’arêtes interdites, il est généralement non-approximable sans hypothèse forte (P ≠ NP).

### Conclusion

Ce problème est :

- NP-difficile,
- de nature combinatoire,
- difficile à approximer.

Il nécessite des algorithmes exacts ou des heuristiques avancées.


## Ressources en ligne

Voici une sélection de ressources en français utiles pour approfondir les notions du projet :

### Problème du Voyageur de Commerce (PVC / TSP)

- Wikipedia : [https://fr.wikipedia.org/wiki/Probl%C3%A8me_du_voyageur_de_commerce](https://fr.wikipedia.org/wiki/Probl%C3%A8me_du_voyageur_de_commerce)

### Problème de tournées de véhicules (VRP)

- Wikipedia : [https://fr.wikipedia.org/wiki/Probl%C3%A8me_de_tourn%C3%A9es_de_v%C3%A9hicules](https://fr.wikipedia.org/wiki/Probl%C3%A8me_de_tourn%C3%A9es_de_v%C3%A9hicules)

### Complexité et NP-complétude

- Wikipedia : [https://fr.wikipedia.org/wiki/Th%C3%A9orie_de_la_complexit%C3%A9_(informatique_th%C3%A9orique)](https://fr.wikipedia.org/wiki/Th%C3%A9orie_de_la_complexit%C3%A9_(informatique_th%C3%A9orique))
- OpenClassrooms – Forum sur P=NP : [https://openclassrooms.com/forum/sujet/p-np-84552](https://openclassrooms.com/forum/sujet/p-np-84552)

### Algorithmes et heuristiques

- Algorithme de Christofides : [https://fr.wikipedia.org/wiki/Algorithme_de_Christofides](https://fr.wikipedia.org/wiki/Algorithme_de_Christofides)
- Heuristique de Lin-Kernighan : [https://fr.wikipedia.org/wiki/Heuristique_de_Lin-Kernighan](https://fr.wikipedia.org/wiki/Heuristique_de_Lin-Kernighan)
- Méthode 2-opt : [https://fr.wikipedia.org/wiki/2-opt](https://fr.wikipedia.org/wiki/2-opt)
